In [ ]:
!pip install praw

In [ ]:
import praw
import pandas as pd

In [ ]:
import json

secret =   {'client_id':"zBSC5ziXXikuqD_PpmYCzw",
             'client_secret':"JXrpx0UErzYqpsmqCOXgZeDBZ1GLuA",
             'password':"duyichen131",
             'user_agent':"attitude towards ai",
             'username':"zimm_yichen",
            }

with open("secret.json", "w") as f:
       json.dump(secret, f)

In [ ]:
with open("secret.json", "r") as f:
    secret = json.load(f)

In [ ]:
reddit = praw.Reddit(
    client_id = secret['client_id'],
    client_secret = secret['client_secret'],
    username = secret['username'],
    password = secret['password'],
    user_agent = secret['user_agent']
)

In [ ]:
def fetch_posts_from_subreddit(subreddit_name, limit=1000):
    subreddit = reddit.subreddit(subreddit_name)
    post_list = []
    keyword = ['AI', 'AIs', 'artificial intelligence', 'Machine Learning', 'llm']
    counter = 1  # Initialize a counter

    for post in subreddit.top(limit=limit):
        for kw in keyword:
            if kw.lower() in post.title.lower() or kw.lower() in post.selftext.lower():
                # Check if post.author is not None before accessing .name
                author_name = post.author.name if post.author else "Unknown"
                post_info = {
                    'id': counter,  # Add the counter as an 'id'
                    'title': post.title,
                    'content': post.selftext,
                    'created_utc': post.created_utc,
                    'author': author_name,  # Use the author_name variable
                    'url': post.url  # Add post's URL
                }
                post_list.append(post_info)
                counter += 1  # Increment the counter
    return post_list

In [ ]:
singularity_posts = fetch_posts_from_subreddit('singularity', limit=1000)
aiwars_posts = fetch_posts_from_subreddit('aiwars', limit=200)

In [ ]:
#将两个社区的帖子合并到一个csv
combined_posts = singularity_posts + aiwars_posts

# 将合并后的帖子列表转换成DataFrame
df_combined = pd.DataFrame(combined_posts)

# 将'created_utc'列转换成日期格式
df_combined['created_utc'] = pd.to_datetime(df_combined['created_utc'], utc=True, unit='s').dt.date

# 导出到CSV，不包含索引列
df_combined.to_csv('combined_posts.csv', index=False)


In [ ]:
non_empty_contents_count = df_combined['content'].replace('', pd.NA).dropna().shape[0]

print(f'Numner of Ccntents: {non_empty_contents_count}')

In [ ]:
import prawcore
from datetime import datetime

def is_valid_reddit_url(url):
    # 检查URL是否是有效的Reddit帖子URL
    return url.startswith('https://www.reddit.com/r/') or url.startswith('https://reddit.com/r/')

def expand_comments(submission):
    # 将所有MoreComments对象替换为实际的评论对象
    submission.comments.replace_more(limit=0)

combined_comments = []

for index, post_info in df_combined.iterrows():
    url = post_info['url']
    if not is_valid_reddit_url(url):
        continue  # 跳过这个URL，继续下一个

    try:
        # 使用praw获取submission对象
        submission_id = url.split('/')[-3]
        submission = reddit.submission(id=submission_id)

        # 展开评论
        expand_comments(submission)

        # 收集评论信息
        for comment in submission.comments.list():
            combined_comments.append({
                'post_id': post_info['id'],
                'comment_author': comment.author.name if comment.author else 'Deleted',
                'comment_body': comment.body,
                'comment_created_utc': datetime.utcfromtimestamp(comment.created_utc).isoformat()
            })
    except prawcore.exceptions.NotFound:
        pass
    except Exception as e:
        pass
print(f'总共爬取了 {len(combined_comments)} 条评论。')

df_all_comments = pd.DataFrame(combined_comments)
df_all_comments.to_csv('combined_comments.csv', index=False)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp combined_posts.csv "/content/drive/My Drive/Colab Notebooks"
!cp combined_comments.csv "/content/drive/My Drive/Colab Notebooks"


In [ ]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction import text
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
posts = pd.read_csv('/content/drive/My Drive/Colab Notebooks/combined_posts.csv', usecols=['processed_title', 'processed_content'])
comments = pd.read_csv('/content/drive/My Drive/Colab Notebooks/combined_comments.csv', usecols=['processed_comment_body'])

posts['text'] = posts['processed_title'] + " " + posts['processed_content']
comments.rename(columns={'processed_comment_body': 'text'}, inplace=True)

data = pd.concat([posts['text'], comments['text']], axis=0).reset_index(drop=True)

data_clean = data.dropna().reset_index(drop=True)

In [ ]:
my_stop_words = list(text.ENGLISH_STOP_WORDS.union(['ai','think','may','thing','way','want','like','use','know','make']))
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words=my_stop_words)
data_vectorized = vectorizer.fit_transform(data_clean)

# 应用 LDA
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(data_vectorized)

# 文档主题分布
document_topics = lda.transform(data_vectorized)

# 选择每个文档的主要主题
topics = document_topics.argmax(axis=1)

# 将主题编号加入到 DataFrame 中
data_clean = pd.DataFrame(data_clean)  # 确保 data_clean 是 DataFrame
data_clean['Topic'] = topics

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\nTopic {topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

print("\nLDA Model Topics and their Keywords:")
display_topics(lda, vectorizer.get_feature_names_out(), 10)

In [ ]:
!pip install gensim

from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

# Define custom stop words
custom_stop_words = ['think','may','thing','way','want','like','use','know','make']

# Tokenize and remove stop words
data_tokenized = [[word for word in str(text).lower().split() if word not in custom_stop_words] for text in data]

# Create a Gensim dictionary
dictionary = Dictionary(data_tokenized)

# Create a Gensim corpus
corpus = [dictionary.doc2bow(text) for text in data_tokenized]

# Use Gensim's LDA
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, random_state=42, passes=20)

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_tokenized, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
doc_topic_dist = [lda_model.get_document_topics(doc_bow) for doc_bow in corpus]

# Select your topic of interest
selected_topic = 2
topic_relevance = []

# Extract the relevance of the selected topic for each document
for doc_distribution in doc_topic_dist:
    doc_topic_relevance = next((score for topic_id, score in doc_distribution if topic_id == selected_topic), 0)
    topic_relevance.append(doc_topic_relevance)

# Sort documents by their relevance to the selected topic
import numpy as np
topic_relevance = np.array(topic_relevance)
top_doc_indices = topic_relevance.argsort()[::-1][:10]  # Indices of the top 10 documents

# Print the most relevant documents to the selected topic
print(f"Top documents for Topic {selected_topic}:\n")
for index in top_doc_indices:
    print(f"Document {index}:\n{data.iloc[index]}\n")
    print("--------------------------------------------------\n")

In [ ]:
analyzer = SentimentIntensityAnalyzer()

def analyze_sentiment_vader(text):
    return analyzer.polarity_scores(text)['compound']

data_clean['Sentiment_Score'] = data_clean['text'].apply(analyze_sentiment_vader)

average_sentiment_per_topic = data_clean.groupby('Topic')['Sentiment_Score'].mean()
print(average_sentiment_per_topic)

In [ ]:
colors = ['#a1c9f4', '#ffb3ba', '#baffc9', '#f8b878', '#f0b7a4']

for i, topic_number in enumerate(sorted(data_clean['Topic'].unique())):
    plt.figure(figsize=(10, 6))
    subset = data_clean[data_clean['Topic'] == topic_number]

    sns.histplot(subset['Sentiment_Score'], kde=False, bins=30, color=colors[i])
    plt.title(f'Topic {topic_number+1} Sentiment Distribution')
    plt.xlabel('Sentiment Score')
    plt.ylabel('Count')
    plt.show()

In [ ]:
plt.figure(figsize=(10, 6))

for topic_number in sorted(data_clean['Topic'].unique()):
    subset = data_clean[data_clean['Topic'] == topic_number]

    sns.kdeplot(subset['Sentiment_Score'], label=f'Topic {topic_number}', bw_adjust=0.5)

plt.title('Sentiment Score Density by Topic')
plt.xlabel('Sentiment Score')
plt.ylabel('Density')
plt.legend()
plt.show()